##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Clasificación de imágenes con Model Maker de TensorFlow Lite

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/lite/models/modify/model_maker/image_classification"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/lite/models/modify/model_maker/image_classification.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/lite/models/modify/model_maker/image_classification.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fuente en GitHub</a>
</td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/lite/models/modify/model_maker/image_classification.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar el bloc de notas</a></td>
  <td>     <a href="https://tfhub.dev/google/imagenet/inception_v3/feature_vector/1"><img src="https://www.tensorflow.org/images/hub_logo_32px.png">Ver modelo de TF Hub</a>
</td>
</table>

La librería [Model Maker de TensorFlow Lite](https://www.tensorflow.org/lite/models/modify/model_maker) simplifica el proceso de adaptación y conversión de un modelo de red neuronal TensorFlow a unos datos de entrada concretos cuando se implementa este modelo para aplicaciones de ML en dispositivos.

Este bloc muestra un ejemplo de principio a fin que usa esta librería Model Maker para ilustrar la adaptación y conversión de un modelo de clasificación de imágenes de uso común para clasificar flores en un dispositivo móvil.

## Requisitos previos

Para ejecutar este ejemplo, primero necesitamos instalar varios paquetes necesarios, incluyendo el paquete Model Maker que se encuentra en [repo](https://github.com/tensorflow/examples/tree/master/tensorflow_examples/lite/model_maker) de GitHub .

In [ ]:
!sudo apt -y install libportaudio2
!pip install -q tflite-model-maker

Importe los paquetes necesarios.

In [ ]:
import os

import numpy as np

import tensorflow as tf
assert tf.__version__.startswith('2')

from tflite_model_maker import model_spec
from tflite_model_maker import image_classifier
from tflite_model_maker.config import ExportFormat
from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.image_classifier import DataLoader

import matplotlib.pyplot as plt

## Ejemplo sencillo de principio a fin

### Obtener la ruta de datos

Tomemos algunas imágenes para jugar con este sencillo ejemplo de principio a fin. Con cientos de imágenes se puede empezar bien con el Model Maker, mientras que con más datos se podría conseguir una mayor precisión.

In [ ]:
image_path = tf.keras.utils.get_file(
      'flower_photos.tgz',
      'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
      extract=True)
image_path = os.path.join(os.path.dirname(image_path), 'flower_photos')

Podría sustituir `image_path` por sus propias carpetas de imágenes. Respecto a subir datos a colab, puede encontrar el botón de subir en la barra lateral izquierda mostrada en la imagen de abajo con el rectángulo rojo. Pruebe a subir un archivo zip y descomprímalo. La ruta raíz del archivo es la ruta actual.

<img src="https://storage.googleapis.com/download.tensorflow.org/models/tflite/screenshots/model_maker_image_classification.png" width="800" hspace="100" alt="Subir archivo">

Si prefiere no subir sus imágenes a la nube, puede intentar ejecutar la librería localmente siguiendo la [guía](https://github.com/tensorflow/examples/tree/master/tensorflow_examples/lite/model_maker) en GitHub.

### Ejecutar el ejemplo

El ejemplo sólo consta de 4 líneas de código, como se muestra a continuación, cada una de las cuales representa un paso del proceso global.


Paso 1. Cargue los datos de entrada específicos de una app de ML en el dispositivo. Divídalos en datos de entrenamiento y datos de prueba.

In [ ]:
data = DataLoader.from_folder(image_path)
train_data, test_data = data.split(0.9)

Paso 2. Personalice el modelo TensorFlow.

In [ ]:
model = image_classifier.create(train_data)

Paso 3. Evalúe el modelo.

In [ ]:
loss, accuracy = model.evaluate(test_data)

Paso 4.  Exporte al modelo TensorFlow Lite.

Aquí, exportamos el modelo TensorFlow Lite con [metadatos](https://www.tensorflow.org/lite/models/convert/metadata) que ofrece un estándar para las descripciones del modelo. El archivo de etiquetas está incrustado en los metadatos. La técnica de cuantización predeterminada tras el entrenamiento es la cuantización entera completa para la tarea de clasificación de imágenes.

Puede descargarla en la barra lateral izquierda al igual que la parte de carga para su propio uso.

In [ ]:
model.export(export_dir='.')

Después de estos sencillos 4 pasos, podríamos seguir usando el archivo de modelo de TensorFlow Lite en aplicaciones en el dispositivo como en la app de referencia [clasificacion de imágenes](https://github.com/tensorflow/examples/tree/master/lite/examples/image_classification).

## Proceso detallado

Actualmente, admitimos varios modelos como EfficientNet-Lite*, MobileNetV2, ResNet50 a modo de modelos preentrenados para la clasificación de imágenes. Pero resulta muy flexible añadir nuevos modelos preentrenados a esta librería con sólo unas pocas líneas de código.

En esta sección se recorre paso a paso este ejemplo de extremo a extremo para ver más detalles.

### Paso 1: Cargue los datos de entrada específicos de una app de ML en el dispositivo

El conjunto de datos de flores contiene 3670 imágenes correspondientes a 5 clases. Descargue la versión de archivo del conjunto de datos y descomprímala.

El conjunto de datos tiene la siguiente estructura de directorios:

<pre>
&lt;b&gt;flower_photos&lt;/b&gt;
|__ &lt;b&gt;daisy&lt;/b&gt;
    |______ 100080576_f52e8ee070_n.jpg
    |______ 14167534527_781ceb1b7a_n.jpg
    |______ ...
|__ &lt;b&gt;dandelion&lt;/b&gt;
    |______ 10043234166_e6dd915111_n.jpg
    |______ 1426682852_e62169221f_m.jpg
    |______ ...
|__ &lt;b&gt;roses&lt;/b&gt;
    |______ 102501987_3cdb8e5394_n.jpg
    |______ 14982802401_a3dfb22afb.jpg
    |______ ...
|__ &lt;b&gt;sunflowers&lt;/b&gt;
    |______ 12471791574_bb1be83df4.jpg
    |______ 15122112402_cafa41934f.jpg
    |______ ...
|__ &lt;b&gt;tulips&lt;/b&gt;
    |______ 13976522214_ccec508fe7.jpg
    |______ 14487943607_651e8062a1_m.jpg
    |______ ...
</pre>

In [ ]:
image_path = tf.keras.utils.get_file(
      'flower_photos.tgz',
      'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
      extract=True)
image_path = os.path.join(os.path.dirname(image_path), 'flower_photos')

Usar la clase `DataLoader` para cargar los datos.

En cuanto al método `from_folder()`, podría cargar los datos desde la carpeta. Se supone que los datos de imagen de la misma clase están en el mismo subdirectorio y el nombre de la subcarpeta es el nombre de la clase. Actualmente, se admiten imágenes codificadas en JPEG y en PNG.

In [ ]:
data = DataLoader.from_folder(image_path)

Divídalo en datos de entrenamiento (80%), datos de validación (10%, opcional) y datos de prueba (10%).

In [ ]:
train_data, rest_data = data.split(0.8)
validation_data, test_data = rest_data.split(0.5)

Muestre 25 ejemplos de imágenes con etiquetas.

In [ ]:
plt.figure(figsize=(10,10))
for i, (image, label) in enumerate(data.gen_dataset().unbatch().take(25)):
  plt.subplot(5,5,i+1)
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  plt.imshow(image.numpy(), cmap=plt.cm.gray)
  plt.xlabel(data.index_to_label[label.numpy()])
plt.show()

### Paso 2: Personalice el modelo TensorFlow

Cree un modelo clasificador de imágenes personalizado basado en los datos cargados. El modelo predeterminado es EfficientNet-Lite0.


In [ ]:
model = image_classifier.create(train_data, validation_data=validation_data)

Vea la estructura detallada del modelo.

In [ ]:
model.summary()

### Paso 3: Evalúe el modelo personalizado

Evalúe el resultado del modelo, obtenga la pérdida y la precisión del modelo.

In [ ]:
loss, accuracy = model.evaluate(test_data)

Podemos representar los resultados predichos en 100 imágenes de prueba. Las etiquetas con color rojo son los resultados predichos erróneos, mientras que los demás son correctos.

In [ ]:
# A helper function that returns 'red'/'black' depending on if its two input
# parameter matches or not.
def get_label_color(val1, val2):
  if val1 == val2:
    return 'black'
  else:
    return 'red'

# Then plot 100 test images and their predicted labels.
# If a prediction result is different from the label provided label in "test"
# dataset, we will highlight it in red color.
plt.figure(figsize=(20, 20))
predicts = model.predict_top_k(test_data)
for i, (image, label) in enumerate(test_data.gen_dataset().unbatch().take(100)):
  ax = plt.subplot(10, 10, i+1)
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  plt.imshow(image.numpy(), cmap=plt.cm.gray)

  predict_label = predicts[i][0][0]
  color = get_label_color(predict_label,
                          test_data.index_to_label[label.numpy()])
  ax.xaxis.label.set_color(color)
  plt.xlabel('Predicted: %s' % predict_label)
plt.show()

Si la precisión no cumple los requisitos de la app, cabría remitirse al [Uso avanzado](#scrollTo=zNDBP2qA54aK) para buscar alternativas, como cambiar a un modelo más grande, ajustar los parámetros de reentrenamiento, etc.

### Paso 4: Exporte a un modelo TensorFlow Lite

Convierta el modelo entrenado al formato de modelo TensorFlow Lite con [metadatos](https://www.tensorflow.org/lite/models/convert/metadata) para poder usarlo posteriormente en una aplicación ML en el dispositivo. El archivo de etiquetas y el archivo de vocabulario están incrustados en los metadatos. El nombre de archivo TFLite predeterminado es `model.tflite`.

En muchas aplicaciones de ML en el dispositivo, el tamaño del modelo es un factor importante. Por lo tanto, se recomienda aplicar la cuantización del modelo para hacerlo más pequeño y, potencialmente, ejecutarlo más rápido. La técnica de cuantización predeterminada tras el entrenamiento es la cuantización total para la tarea de clasificación de imágenes.

In [ ]:
model.export(export_dir='.')

Vea la [guía de ejemplo](https://www.tensorflow.org/lite/examples/image_classification/overview) de clasificación de imágenes para saber más sobre cómo integrar el modelo TensorFlow Lite en apps móviles.

Este modelo puede integrarse en una app Android o iOS usando la [API de Clasificador de imágenes](https://www.tensorflow.org/lite/inference_with_metadata/task_library/image_classifier) de la [TensorFlow Lite Task Library](https://www.tensorflow.org/lite/inference_with_metadata/task_library/overview).

Los formatos de exportación permitidos pueden ser uno o varios de los siguientes:

- `ExportFormat.TFLITE`
- `ExportFormat.LABEL`
- `ExportFormat.SAVED_MODEL`

De forma predeterminada, sólo exporta el modelo TensorFlow Lite con metadatos. También puede exportar selectivamente diferentes archivos. Por ejemplo, se puede exportar sólo el archivo de etiquetas de la siguiente manera:

In [ ]:
model.export(export_dir='.', export_format=ExportFormat.LABEL)

También puede evaluar el modelo tflite con el método `evaluate_tflite`.

In [ ]:
model.evaluate_tflite('model.tflite', test_data)

## Uso avanzado

La función `create` es la parte crítica de esta librería. Usa el aprendizaje por transferencia con un modelo preentrenado similar al del [tutorial](https://www.tensorflow.org/tutorials/images/transfer_learning).

La función `create` contiene los siguientes pasos:

1. Divida los datos en datos de entrenamiento, validación y prueba según los parámetros `validation_ratio` y `test_ratio`. El valor predeterminado de `validation_ratio` y `test_ratio` son `0.1` y `0.1`.
2. Descargue un [Vector de características de imagen](https://www.tensorflow.org/hub/common_signatures/images#image_feature_vector) como modelo base desde TensorFlow Hub. El modelo predeterminado preentrenado es EfficientNet-Lite0.
3. Añada una cabecera del clasificador con una capa Dropout con `dropout_rate` entre la capa de cabecera y el modelo preentrenado. El valor predeterminado `dropout_rate` es el valor predeterminado `dropout_rate` de [make_image_classifier_lib](https://github.com/tensorflow/hub/blob/master/tensorflow_hub/tools/make_image_classifier/make_image_classifier_lib.py#L55) de TensorFlow Hub.
4. Preprocesar los datos de entrada en bruto. Actualmente, los pasos del preprocesamiento incluyen normalizar el valor de cada pixel de la imagen para modelar la escala de entrada y redimensionarla para modelar el tamaño de entrada. EfficientNet-Lite0 tiene la escala de entrada `[0, 1]` y el tamaño de la imagen de entrada `[224, 224, 3]`.
5. Introduzca los datos en el modelo clasificador. De forma predeterminada, los parámetros de entrenamiento, como épocas de entrenamiento, tamaño del lote, tasa de aprendizaje, impulso, son los valores predeterminados de [make_image_classifier_lib](https://github.com/tensorflow/hub/blob/master/tensorflow_hub/tools/make_image_classifier/make_image_classifier_lib.py#L55) de TensorFlow Hub. Sólo se entrena la cabecera del clasificador.

En esta sección describimos varios temas avanzados, como el cambio a un modelo de clasificación de imágenes diferente, la modificación de los hiperparámetros de entrenamiento, etc.


## Personalice la cuantización posterior al entrenamiento en el modelo TensorFLow Lite


[La cuantización posterior al entrenamiento](https://www.tensorflow.org/lite/performance/post_training_quantization) es una técnica de conversión que puede reducir el tamaño del modelo y la latencia de la inferencia, al tiempo que mejora la velocidad de inferencia de la CPU y del acelerador de hardware, con una pequeña degradación de la precisión del modelo. Por ello, se usa mucho para optimizar el modelo.


La librería Model Maker aplica una técnica predeterminada de cuantización postentrenamiento al exportar el modelo. Si desea personalizar la cuantización postentrenamiento, Model Maker también soporta múltiples opciones de cuantización postentrenamiento utilizando [QuantizationConfig](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker/config/QuantizationConfig). Tomemos como ejemplo la cuantización float16. En primer lugar, defina la configuración de cuantización.

In [ ]:
config = QuantizationConfig.for_float16()

A continuación, exportamos el modelo TensorFlow Lite con dicha configuración.

In [ ]:
model.export(export_dir='.', tflite_filename='model_fp16.tflite', quantization_config=config)

En Colab, puede descargar el modelo llamado `model_fp16.tflite` de la barra lateral izquierda, igual que la parte de carga mencionada anteriormente.

## Cambiar el modelo


### Cambie al modelo compatible con esta librería.

Esta librería ya es compatible con los modelos EfficientNet-Lite, MobileNetV2 y ResNet50. [EfficientNet-Lite](https://github.com/tensorflow/tpu/tree/master/models/official/efficientnet/lite) son una familia de modelos de clasificación de imágenes que pueden alcanzar una precisión de última generación y son adecuados para dispositivos Edge. El modelo predeterminado es EfficientNet-Lite0.

Podríamos cambiar de modelo a MobileNetV2 con sólo configurar el parámetro `model_spec` a la especificación del modelo MobileNetV2 en el método `create`.

In [ ]:
model = image_classifier.create(train_data, model_spec=model_spec.get('mobilenet_v2'), validation_data=validation_data)

Evalúe el modelo MobileNetV2 recién reentrenado para ver la precisión y la pérdida en los datos de prueba.

In [ ]:
loss, accuracy = model.evaluate(test_data)

### Cambie al modelo en TensorFlow Hub

Además, también podríamos pasar a otros modelos nuevos que introducen una imagen y emiten un vector de características con formato TensorFlow Hub.

Tomando como ejemplo el modelo [Inception V3](https://tfhub.dev/google/imagenet/inception_v3/feature_vector/1), podríamos definir `inception_v3_spec` que es un objeto de [image_classifier.ModelSpec](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker/image_classifier/ModelSpec) y contiene la especificación del modelo Inception V3.

Necesitamos especificar el nombre del modelo `name`, la url del modelo TensorFlow Hub `uri`. Mientras tanto, el valor predeterminado de `input_image_shape` es `[224, 224]`. Tenemos que cambiarlo a `[299, 299]` para el modelo Inception V3.

In [ ]:
inception_v3_spec = image_classifier.ModelSpec(
    uri='https://tfhub.dev/google/imagenet/inception_v3/feature_vector/1')
inception_v3_spec.input_image_shape = [299, 299]

Entonces, configurando el parámetro `model_spec` como `inception_v3_spec` en el método `create`, podríamos volver a entrenar el modelo Inception V3.

Los pasos restantes son exactamente iguales y al final podremos tener un modelo InceptionV3 TensorFlow Lite personalizado.

### Cambie su propio modelo personalizado

Si queremos usar el modelo personalizado que no está en TensorFlow Hub, debemos crear y exportar [ModelSpec](https://www.tensorflow.org/hub/api_docs/python/hub/ModuleSpec) en TensorFlow Hub.

A continuación, empiece a definir el objeto `ModelSpec` como en el proceso anterior.

## Cambie los hiperparámetros de entrenamiento

También podemos cambiar los hiperparámetros de entrenamiento como `epochs`, `dropout_rate` y `batch_size` que pueden afectar a la precisión del modelo. Los parámetros del modelo que puede ajustar son:

- `epochs`: con más épocas se podría lograr una mayor precisión hasta que converja, pero un entrenamiento para demasiadas épocas puede llevar a un sobreajuste.
- `dropout_rate`: La tasa de abandono, para evitar el sobreajuste. Ninguno, de forma predeterminada.
- `batch_size`: Número de muestras a usar en un paso de entrenamiento. Ninguno, de forma predeterminada.
- `validation_data`: Datos de validación. Si es Ninguno, omite el proceso de validación. Ninguno, de forma predeterminada.
- `train_whole_model`: Si es true, el módulo Hub se entrena junto con la capa de clasificación superior. En caso contrario, sólo se entrena la capa de clasificación superior. Ninguno, de forma predeterminada.
- `learning_rate`: Tasa de aprendizaje base. Ninguno, de forma predeterminada.
- `momentum`: un float Python reenviado al optimizador. Sólo se usa cuando `use_hub_library` es True. Ninguno, de forma predeterminada.
- `shuffle`: Booleano, si los datos deben mezclarse. False, de forma predeterminada.
- `use_augmentation`: Booleano, use la aumentación de datos para el preprocesamiento. False, de forma predeterminada.
- `use_hub_library`: Booleano, use `make_image_classifier_lib` de tensorflow hub para volver a entrenar el modelo. Esta canalización del entrenamiento podría lograr un mejor rendimiento para conjuntos de datos complicados con muchas categorías. True, de forma predeterminada.
- `warmup_steps`: Número de pasos de calentamiento para el programa de calentamiento en la tasa de aprendizaje. Si es Ninguno, se usa el predeterminado warmup_steps que es el total de pasos de entrenamiento en dos épocas. Sólo se usa cuando `use_hub_library` es False. Ninguno, de forma predeterminada.
- `model_dir`: Opcional, la ubicación de los archivos de punto de verificación del modelo. Sólo se usa cuando `use_hub_library` es False. Ninguno, de forma predeterminada.

Los parámetros que son None de forma predeterminada como `epochs` recibirán los parámetros concretos predeterminados en [make_image_classifier_lib](https://github.com/tensorflow/hub/blob/02ab9b7d3455e99e97abecf43c5d598a5528e20c/tensorflow_hub/tools/make_image_classifier/make_image_classifier_lib.py#L54) de la librería TensorFlow Hub o [train_image_classifier_lib](https://github.com/tensorflow/examples/blob/f0260433d133fd3cea4a920d1e53ecda07163aee/tensorflow_examples/lite/model_maker/core/task/train_image_classifier_lib.py#L61).

Por ejemplo, podríamos entrenar con más épocas.


In [ ]:
model = image_classifier.create(train_data, validation_data=validation_data, epochs=10)

Evalúe el modelo recién reentrenado con 10 épocas de entrenamiento.

In [ ]:
loss, accuracy = model.evaluate(test_data)

# Más información

Puede leer nuestro ejemplo de [clasificación de imágenes](https://www.tensorflow.org/lite/examples/image_classification/overview) para conocer los detalles técnicos. Para más información, consulte:

- [Guía](https://www.tensorflow.org/lite/models/modify/model_maker) y [Referencia de API](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker) de Model Maker de TensorFlow Lite.
- Librería de tareas: [ImageClassifier](https://www.tensorflow.org/lite/inference_with_metadata/task_library/image_classifier) para su implementación.
- Las apps de referencia de principio a fin: [Android](https://github.com/tensorflow/examples/tree/master/lite/examples/image_classification/android), [iOS](https://github.com/tensorflow/examples/tree/master/lite/examples/image_classification/ios), y [Raspberry PI](https://github.com/tensorflow/examples/tree/master/lite/examples/image_classification/raspberry_pi).
